# Analyse Crypto

In [12]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import time
import zipfile
from io import BytesIO
import warnings
from ipywidgets import FloatProgress, HTML, VBox
from IPython.display import display, clear_output

# Durée d'analyse (en jours) - Binance Vision est optimisé pour de grandes périodes
DUREE_JOURS = 365

# Intervalle des données
# Options: 1s, 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
INTERVAL = '1h'

# Symbole à analyser
SYMBOL = 'BTCUSDT'

# Date de fin (format YYYY-MM-DD) — par défaut aujourd'hui
from datetime import datetime
END_DATE = None

if END_DATE == None:
  END_DATE = datetime.now().strftime('%Y-%m-%d')

print(f"Configuration:")
print(f"- Symbole: {SYMBOL}")
print(f"- Durée d'analyse: {DUREE_JOURS} jours")
print(f"- Intervalle: {INTERVAL}")
print(f"- Date de fin: {END_DATE}")
print(f"- Source: Binance Vision (données historiques)")
print()

warnings.filterwarnings('ignore')

def download_binance_vision_data(symbol=SYMBOL, interval=INTERVAL, days_back=DUREE_JOURS, end_date_str=None):

    # Résolution de la date de fin
    if end_date_str is None:
        try:
            end_date_str = END_DATE  # variable globale définie dans la cellule de config
        except NameError:
            end_date_str = None

    if end_date_str:
        try:
            end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
        except Exception:
            print(f"Format END_DATE invalide: {end_date_str}. Utilisation de datetime.now().")
            end_date = datetime.now()
    else:
        end_date = datetime.now()

    # Calculer la date de début
    start_date = end_date - timedelta(days=days_back)

    # Construire la liste des dates à télécharger
    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)

    # Création des widgets de progression
    progress_bar = FloatProgress(
        value=0,
        min=0,
        max=len(dates),
        bar_style='info',
        style={'bar_color': '#00c851'},
        orientation='horizontal'
    )
    
    # status_label = HTML(value=f"<b>Préparation du téléchargement de {len(dates)} fichiers...</b>")
    
    # Conteneur pour afficher la barre de progression
    progress_widget = VBox([progress_bar])
    display(progress_widget)

    all_data = []
    success_count = 0
    total_periods = 0

    from concurrent.futures import ThreadPoolExecutor, as_completed

    def fetch_for_date(date_str):
        """Télécharge et retourne un tuple (date_str, df_day or None, error_message or None, periods_count)"""
        url = f"https://data.binance.vision/data/spot/daily/klines/{symbol}/{interval}/{symbol}-{interval}-{date_str}.zip"
        try:
            # Petite trace locale (renvoyée pour agrégation)
            resp = requests.get(url, timeout=30)
            if resp.status_code == 200:
                with zipfile.ZipFile(BytesIO(resp.content)) as zip_file:
                    csv_filename = f"{symbol}-{interval}-{date_str}.csv"
                    if csv_filename in zip_file.namelist():
                        csv_content = zip_file.read(csv_filename)
                        df_day = pd.read_csv(BytesIO(csv_content), header=None, names=[
                            'timestamp', 'open', 'high', 'low', 'close', 'volume',
                            'close_time', 'quote_asset_volume', 'number_of_trades',
                            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
                        ])
                        return (date_str, df_day, None, len(df_day))
                    else:
                        return (date_str, None, 'CSV not found in ZIP', 0)
            else:
                return (date_str, None, f'HTTP {resp.status_code}', 0)
        except requests.exceptions.RequestException:
            return (date_str, None, 'Network error', 0)
        except zipfile.BadZipFile:
            return (date_str, None, 'Bad ZIP', 0)
        except Exception as e:
            return (date_str, None, str(e)[:200], 0)

    # Paramètres de parallélisme: limiter le nombre de threads raisonnablement
    max_workers = min(12, max(4, len(dates)))
    completed_count = 0

    with ThreadPoolExecutor(max_workers=max_workers) as exe:
        future_to_date = {exe.submit(fetch_for_date, d): d for d in dates}
        for fut in as_completed(future_to_date):
            date_str = future_to_date[fut]
            try:
                d, df_day, err, periods = fut.result()
                completed_count += 1
                
                # Mise à jour de la barre de progression
                progress_bar.value = completed_count
                
                if df_day is not None:
                    # --- Normalisation des timestamps: uniformiser en microsecondes (us) ---
                    try:
                        # S'assurer que la colonne 'timestamp' existe et est numérique
                        if 'timestamp' not in df_day.columns:
                            # si les colonnes sont indexées numériquement, la première colonne est le timestamp
                            df_day.rename(columns={0: 'timestamp'}, inplace=True)
                        df_day['timestamp'] = pd.to_numeric(df_day['timestamp'], errors='coerce')
                    except Exception:
                        pass

                    try:
                        # Détecter les timestamps inférieurs au seuil (2025-01-01)
                        threshold_dt = datetime(2025, 1, 1)
                        # Seuil en millisecondes correspondant à 2025-01-01
                        threshold_ms = int(threshold_dt.timestamp() * 1000)

                        # Heuristique: si la majorité des timestamps sont < threshold_ms*10, ils sont probablement en ms
                        max_ts = pd.to_numeric(df_day['timestamp'], errors='coerce').max()
                        if pd.notna(max_ts):
                            # Si le maximum observé est inférieur à threshold_ms * 10, on suppose que les valeurs sont en ms
                            if max_ts < threshold_ms * 10:
                                mask_ms = pd.to_numeric(df_day['timestamp'], errors='coerce') < (threshold_ms * 10)
                                if mask_ms.any():
                                    df_day.loc[mask_ms, 'timestamp'] = pd.to_numeric(df_day.loc[mask_ms, 'timestamp'], errors='coerce') * 1000
                    except Exception as e:
                        print(f"⚠ Erreur lors de la normalisation des timestamps: {e}")

                    all_data.append(df_day)
                    success_count += 1
                    total_periods += periods
                    
            except Exception as e:
                completed_count += 1
                progress_bar.value = completed_count
               
    # Finalisation de la barre de progression
    progress_bar.bar_style = 'success' if success_count > 0 else 'danger'
   
    # Petit délai global pour rester poli si nécessaire
    time.sleep(0.1)

    if all_data:
        # Combiner toutes les données
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df.to_dict('records')
    else:
        print("Aucune donnée récupérée via Binance Vision")
        return []

def create_dataframe(data, symbol_name):
    if not data:
        print(f"Aucune donnée disponible pour {symbol_name}")
        return pd.DataFrame()
        
    df = pd.DataFrame(data, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    
    # Conversion des types
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = df[col].astype(float)
    
    # Gestion des timestamps - Binance Vision utilise différents formats
    timestamp_converted = False
    
    try:
        # Essai avec microsecondes (format attendu après normalisation)
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='us')
        timestamp_converted = True
        # print("✓ Timestamps convertis depuis microsecondes")
    except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
        try:
            # Essai avec millisecondes (fallback)
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            timestamp_converted = True
        except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
            try:
                # Essai avec secondes
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
                timestamp_converted = True
            except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
                try:
                    # Si les valeurs sont trop grandes, diviser par 1000000 (microsecondes vers secondes)
                    df['timestamp'] = pd.to_datetime(df['timestamp'] / 1000000, unit='s')
                    timestamp_converted = True
                except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
                    print("✗ Impossible de convertir les timestamps")

    if not timestamp_converted:
        print(f"✗ Erreur: Impossible de traiter les timestamps pour {symbol_name}")
        return pd.DataFrame()
    
    # Vérifier la plausibilité des dates
    min_date = df['timestamp'].min()
    max_date = df['timestamp'].max()
    
    if min_date.year < 2009 or max_date.year > 2030:
        print(f"⚠ Dates suspectes détectées: {min_date} à {max_date}")
    
    df.set_index('timestamp', inplace=True)
    
    # Supprimer les doublons et trier
    df = df[~df.index.duplicated(keep='first')].sort_index()
    
    return df

# Téléchargement des données
btc_data = download_binance_vision_data(SYMBOL, INTERVAL, DUREE_JOURS)

# Création du DataFrame
df_btc = create_dataframe(btc_data, SYMBOL)

Configuration:
- Symbole: BTCUSDT
- Durée d'analyse: 365 jours
- Intervalle: 1h
- Date de fin: 2025-09-23
- Source: Binance Vision (données historiques)



In [5]:
# VISUALISATION INTERACTIVE

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pywt  # PyWavelets pour l'analyse par ondelettes

# Paramètres configurables
MA_PERIOD = 24  # Période de la moyenne mobile
TOP_FREQUENCIES = 200  # Nombre de pics dominants à garder pour la reconstruction filtrée

if len(df_btc) > 0:
    # Calculer la moyenne mobile sur les MA_PERIOD dernières périodes
    df_btc[f'ma_{MA_PERIOD}'] = df_btc['close'].rolling(window=MA_PERIOD).mean()
    # Décalage vers la gauche pour aligner (la fin des données n'est pas importante)
    decalage = MA_PERIOD // 2
    df_btc[f'ma_{MA_PERIOD}_shifted'] = df_btc[f'ma_{MA_PERIOD}'].shift(-decalage)

    # Calculer la différence entre close et la MA décalée
    df_btc['diff_close_ma'] = df_btc['close'] - df_btc[f'ma_{MA_PERIOD}_shifted']

    # Série de différences (sans NaN)
    diff_series = df_btc['diff_close_ma'].dropna()
    difference_values = diff_series.values

    # --- Création de quatre subplots empilés (prix+volume, diff temporelle, ondelettes, spectre) ---
    # Le premier subplot aura un axe Y secondaire pour le volume
    fig = make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=False,  # On va gérer manuellement la synchronisation
        vertical_spacing=0.05,
        specs=[[{"secondary_y": True}],   # Axe Y secondaire pour le volume
               [{"secondary_y": False}], 
               [{"secondary_y": False}],  # Ondelettes
               [{"secondary_y": False}]],
        subplot_titles=('Prix, Moyenne Mobile & Volume', 'Différence (Close - MA)', 'Ondelettes sur Différence - Basses Fréquences', 'Spectre FFT (Fréquences)')
    )

    # Prix + MA sur la première rangée (row=1) - axe Y primaire
    fig.add_trace(
        go.Scatter(x=df_btc.index, y=df_btc['close'], mode='lines', name=f"{SYMBOL} Close", line=dict(color='#00c851', width=1)),
        row=1,
        col=1,
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df_btc.index, y=df_btc[f'ma_{MA_PERIOD}_shifted'], mode='lines', name=f"MA {MA_PERIOD} (shifted)", line=dict(color='blue', width=1)),
        row=1,
        col=1,
        secondary_y=False,
    )

    # Volume sur la première rangée (row=1) - axe Y secondaire
    fig.add_trace(
        go.Scatter(x=df_btc.index, y=df_btc['volume'], mode='lines', name='Volume', line=dict(color='green', width=1), opacity=0.3),
        row=1,
        col=1,
        secondary_y=True,
    )

    # Différence sur la deuxième rangée (row=2)
    fig.add_trace(
        go.Scatter(x=df_btc.index, y=df_btc['diff_close_ma'], mode='lines', name='Close - MA (shifted)', line=dict(color='red', width=1)),
        row=2,
        col=1,
    )

    # === ANALYSE PAR ONDELETTES MORLET (alternative à Daubechies) ===
    try:
        # Préparer les données pour l'analyse par ondelettes (utiliser le signal de différence)
        # Utiliser la différence Close - MA pour analyser les déviations par rapport à la tendance
        signal = df_btc['diff_close_ma'].dropna().values
        time_index = df_btc['diff_close_ma'].dropna().index
        
        # Réduire le signal pour éviter des calculs trop lourds
        max_signal_length = 1024  # Limiter pour performance
        if len(signal) > max_signal_length:
            step = len(signal) // max_signal_length
            signal = signal[::step]
            time_index = time_index[::step]
        
        # Paramètres pour l'analyse par ondelettes optimisés pour basses fréquences
        wavelet = 'morl'  # Ondelette de Morlet (alternative à Daubechies)
        # Étendre la plage d'échelles vers les grandes valeurs pour capturer les basses fréquences
        scales = np.geomspace(1, 256, 80)  # Plus d'échelles avec extension vers 256
        
        # Calculer la transformée en ondelettes continue (CWT)
        coefficients, frequencies = pywt.cwt(signal, scales, wavelet)
        
        # Calculer les pseudo-fréquences pour Morlet
        # Pour Morlet, la fréquence est approximativement 1/scale
        pseudo_frequencies = 1.0 / scales  # Fréquence inversement proportionnelle à l'échelle
        
        # Créer le scalogramme avec normalisation logarithmique pour améliorer le contraste des basses fréquences
        scalogram = np.abs(coefficients)
        
        # Normalisation logarithmique pour accentuer les faibles amplitudes (basses fréquences)
        # Ajouter une petite constante pour éviter log(0)
        epsilon = np.max(scalogram) * 1e-6
        scalogram_log = np.log10(scalogram + epsilon)
        
        # Filtrer pour se concentrer sur les basses fréquences (échelles > 8, soit fréquences < 0.125)
        low_freq_mask = scales >= 8  # Garder les échelles >= 8
        scales_filtered = scales[low_freq_mask]
        pseudo_frequencies_filtered = pseudo_frequencies[low_freq_mask]
        scalogram_filtered = scalogram_log[low_freq_mask, :]
        
        # Ajouter le scalogramme comme heatmap avec colormap optimisée pour le contraste
        fig.add_trace(
            go.Heatmap(
                x=time_index,
                y=pseudo_frequencies_filtered,
                z=scalogram_filtered,
                colorscale='Plasma',  # Plasma offre un meilleur contraste pour les faibles valeurs
                name='Ondelettes Morlet (Basses Fréq.)',
                showscale=True,
                colorbar=dict(title="Log Amplitude", x=1.02, len=0.2, y=0.4)
            ),
            row=3,
            col=1
        )
        
        print(f"Analyse ondelettes réussie !")
        print(f"Wavelet: {wavelet} (Morlet)")
        print(f"Signal analysé: Différence (Close - MA) pour capturer les déviations")
        print(f"Nombre d'échelles totales: {len(scales)}")
        print(f"Nombre d'échelles affichées (basses fréq.): {len(scales_filtered)}")
        print(f"Points de signal: {len(signal)}")
        print(f"Plage de fréquences affichées: {pseudo_frequencies_filtered.min():.6f} - {pseudo_frequencies_filtered.max():.6f}")
        print(f"Échelles affichées: {scales_filtered.min():.1f} - {scales_filtered.max():.1f}")
        print(f"Normalisation: Logarithmique (log10) pour améliorer contraste basses fréquences")
        
    except Exception as e:
        print(f'Erreur analyse ondelettes: {e}')
        import traceback
        traceback.print_exc()
        # Afficher un message sur le graphique en cas d'erreur
        fig.add_annotation(text=f'Erreur ondelettes: {str(e)[:50]}', xref='paper', yref='paper', x=0.5, y=0.4, showarrow=False, row=3, col=1)

    # Ajout d'une analyse de Fourier (FFT) sur la série de différences
    try:
        N = len(difference_values)
        if N >= 4:
            # Detrend simple: retirer la moyenne
            vals = difference_values - np.mean(difference_values)
            # FFT (real-valued optimisée)
            fft_vals = np.fft.rfft(vals)
            fft_freqs = np.fft.rfftfreq(N, d=1.0)  # cycles per sample
            fft_amp = np.abs(fft_vals)

            # Convertir fréquence -> période (en nombre d'échantillons). On ignore la fréquence 0 (DC) pour la période.
            nonzero = fft_freqs > 0
            periods = np.full_like(fft_freqs, np.nan, dtype=float)
            periods[nonzero] = 1.0 / fft_freqs[nonzero]

            # Préparer données pour affichage (exclure DC et utiliser les fréquences)
            plot_mask = nonzero
            plot_freqs = fft_freqs[plot_mask]  # Utiliser les fréquences au lieu des périodes
            plot_amp = fft_amp[plot_mask]

            # Limiter la plage de fréquence affichée pour correspondre aux ondelettes
            # Garder seulement les basses fréquences similaires aux ondelettes (< 0.125)
            max_freq = 0.5  # Limiter à 0.5 pour avoir un bon détail des basses fréquences
            keep = plot_freqs <= max_freq
            plot_freqs = plot_freqs[keep]
            plot_amp = plot_amp[keep]

            # Tracer le spectre (Amplitude) en fonction de la fréquence - SUR LA 4ème RANGÉE
            fig.add_trace(
                go.Scatter(x=plot_freqs, y=plot_amp, mode='lines', name='FFT Amplitude', line=dict(color='purple', width=1)),
                row=4,
                col=1,
            )

            # Reconstruction avec filtre des top fréquences dominantes
            try:
                # Reconstruction complète (sans filtrage)
                recon_full = np.fft.irfft(fft_vals, n=N)
                recon_full = recon_full + np.mean(difference_values)
                
                # Filtre pour garder seulement les top N pics dominants
                fft_vals_filtered = np.zeros_like(fft_vals)
                # Garder la composante DC (fréquence 0)
                fft_vals_filtered[0] = fft_vals[0]
                
                # Identifier les top N pics (excluant DC)
                if len(fft_amp) > 1:
                    k = min(TOP_FREQUENCIES, len(fft_amp) - 1)  # -1 pour exclure DC
                    # Indices des top k amplitudes (excluant DC à l'index 0)
                    top_idx = np.argsort(fft_amp[1:])[-k:] + 1  # +1 pour compenser l'exclusion de DC
                    # Garder seulement ces fréquences
                    fft_vals_filtered[top_idx] = fft_vals[top_idx]
                
                # Reconstruction filtrée
                recon_filtered = np.fft.irfft(fft_vals_filtered, n=N)
                recon_filtered = recon_filtered + np.mean(difference_values)
                
                # Mesures d'erreur pour les deux reconstructions
                mse_full = np.mean((recon_full - difference_values)**2)
                mse_filtered = np.mean((recon_filtered - difference_values)**2)
                max_err_full = np.max(np.abs(recon_full - difference_values))
                max_err_filtered = np.max(np.abs(recon_filtered - difference_values))
                
                # Tracer la reconstruction filtrée (top N)
                fig.add_trace(
                    go.Scatter(x=diff_series.index, y=recon_filtered, mode='lines', name=f'Reconstruction top {TOP_FREQUENCIES}', line=dict(color='orange', width=1)),
                    row=2, col=1
                )
                
                print(f"FFT - Reconstruction complète - MSE: {mse_full:.3e}, Max Error: {max_err_full:.3e}")
                print(f"FFT - Reconstruction top {TOP_FREQUENCIES} - MSE: {mse_filtered:.3e}, Max Error: {max_err_filtered:.3e}")
                
            except Exception as e:
                print(f"Erreur reconstruction: {e}")

            # Annoter les pics dominants (top N) pour aider l'interprétation - SUR LA 4ème RANGÉE
            try:
                k = min(5, len(plot_amp))  # Top 5 seulement pour les annotations
                top_idx = np.argsort(plot_amp)[-k:][::-1]
                top_freqs = plot_freqs[top_idx]  # Utiliser les fréquences
                top_amp = plot_amp[top_idx]
                # Ajouter des marqueurs et labels
                fig.add_trace(
                    go.Scatter(x=top_freqs, y=top_amp, mode='markers+text', text=[f"{f:.3f}" for f in top_freqs], textposition='top center', marker=dict(color='red', size=6), showlegend=False),
                    row=4, col=1
                )
            except Exception:
                pass
        else:
            # Trop peu de points pour une FFT fiable - SUR LA 4ème RANGÉE
            fig.add_annotation(text='N trop petit pour FFT', xref='paper', yref='paper', x=0.5, y=0.05, showarrow=False, row=4, col=1)
    except Exception as e:
        print('Erreur FFT:', e)

    # Ajouter une ligne horizontale fine et pointillée (y=0) sur la rangée des différences (row=2)
    try:
        fig.add_hline(y=0, line=dict(color='gray', dash='dot', width=1), row=2, col=1)
    except Exception:
        try:
            fig.add_shape(
                type='line',
                x0=df_btc.index.min(), x1=df_btc.index.max(),
                y0=0, y1=0,
                xref='x', yref='y',
                line=dict(color='gray', dash='dot', width=1),
                row=2, col=1,
            )
        except Exception:
            pass

    # Layout improvements
    fig.update_layout(
        height=1600,  # Augmenté pour 4 graphiques
        showlegend=False,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
        hovermode='x unified',
        title_text=f"{SYMBOL} ({DUREE_JOURS}d @ {INTERVAL}) - Close, MA {MA_PERIOD}, Volume, Diff, Ondelettes & FFT", 
        margin=dict(l=60, r=60, t=80, b=120),
    )

    # Axis labels: row 1 = Price (primaire) + Volume (secondaire), row 2 = Diff, row 3 = Ondelettes, row 4 = FFT
    fig.update_yaxes(title_text='Price', row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text='Volume', row=1, col=1, secondary_y=True)
    fig.update_yaxes(title_text='Diff', row=2, col=1)
    fig.update_yaxes(title_text='Fréquence (1/échelle) - Focus Basses Fréq.', row=3, col=1, type='log')  # Échelle logarithmique pour les fréquences
    fig.update_yaxes(title_text='Amplitude', row=4, col=1)

    # Configure X-axis appearance for datetime subplot
    date_xargs = dict(type='date', tickformat="%Y-%m-%d\n%H:%M", tickangle=-45, tickfont=dict(size=10), nticks=8, ticks='outside', showgrid=False, showticklabels=True, title_standoff=20)
    try:
        x0 = df_btc.index.min()
        x1 = df_btc.index.max()
        
        # Synchroniser les axes X des trois premiers graphiques (row 1, 2 et 3)
        # Les trois auront la même plage et seront liés pour le zoom/pan
        fig.update_xaxes(range=[x0, x1], title_text='', matches='x3', **date_xargs, row=1, col=1)
        fig.update_xaxes(range=[x0, x1], title_text='', matches='x3', **date_xargs, row=2, col=1)
        fig.update_xaxes(range=[x0, x1], title_text='Time', **date_xargs, row=3, col=1)
        
    except Exception:
        fig.update_xaxes(title_text='Time', **date_xargs, row=3, col=1)

    # X-axis pour le spectre: fréquence (même unité que l'axe Y des ondelettes) - SUR LA 4ème RANGÉE
    try:
        fig.update_xaxes(title_text='Fréquence (cycles/échantillon)', row=4, col=1, type='log')
    except Exception:
        pass

    # Affichage simple
    try:
        fig.show(renderer='vscode')
    except Exception as e:
        print("Erreur: impossible d'afficher la figure avec le renderer 'vscode'.")
        print("Détail: ", str(e))

Analyse ondelettes réussie !
Wavelet: morl (Morlet)
Signal analysé: Différence (Close - MA) pour capturer les déviations
Nombre d'échelles totales: 80
Nombre d'échelles affichées (basses fréq.): 50
Points de signal: 1093
Plage de fréquences affichées: 0.003906 - 0.121753
Échelles affichées: 8.2 - 256.0
Normalisation: Logarithmique (log10) pour améliorer contraste basses fréquences
FFT - Reconstruction complète - MSE: 2.893e-25, Max Error: 3.638e-12
FFT - Reconstruction top 200 - MSE: 2.020e+05, Max Error: 3.183e+03
